In [60]:
import numpy as np
import pandas as pd
import math
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder

Считать тренировочный набор данных из файла train.csv, просмотреть размерность полученного DataFrame, информацию о столбцах и их типах, количестве непустых записей. Посмотреть распределение целевой переменной

In [97]:
data = pd.read_csv('train.csv', sep=',')
data.head(5)

,line num,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


In [98]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 25 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   line num                           103904 non-null  int64  
 1   id                                 103904 non-null  int64  
 2   Gender                             103904 non-null  object 
 3   Customer Type                      103904 non-null  object 
 4   Age                                103904 non-null  int64  
 5   Type of Travel                     103904 non-null  object 
 6   Class                              103904 non-null  object 
 7   Flight Distance                    103904 non-null  int64  
 8   Inflight wifi service              103904 non-null  int64  
 9   Departure/Arrival time convenient  103904 non-null  int64  
 10  Ease of Online booking             103904 non-null  int64  
 11  Gate location                      1039

In [99]:
nullrows = data[data['Arrival Delay in Minutes'].isnull()].index
for idx in nullrows:
    data.loc[idx, 'Arrival Delay in Minutes'] = data.loc[idx, 'Departure Delay in Minutes']

Отобразить все столбцы с типом object

In [100]:
X2 = data.select_dtypes(include=[object])
X2.head(30)

,Gender,Customer Type,Type of Travel,Class,satisfaction
0,Male,Loyal Customer,Personal Travel,Eco Plus,neutral or dissatisfied
1,Male,disloyal Customer,Business travel,Business,neutral or dissatisfied
2,Female,Loyal Customer,Business travel,Business,satisfied
3,Female,Loyal Customer,Business travel,Business,neutral or dissatisfied
4,Male,Loyal Customer,Business travel,Business,satisfied
5,Female,Loyal Customer,Personal Travel,Eco,neutral or dissatisfied
6,Male,Loyal Customer,Personal Travel,Eco,neutral or dissatisfied
7,Female,Loyal Customer,Business travel,Business,satisfied
8,Female,Loyal Customer,Business travel,Business,neutral or dissatisfied
9,Male,disloyal Customer,Business travel,Eco,neutral or dissatisfied


Кодируем целевую переменную, результат сохраняем в массив y

In [43]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y = le.fit_transform(data['satisfaction'])
print(list(le.classes_))
print(y)

['neutral or dissatisfied', 'satisfied']
[0 0 1 ... 0 0 0]


Список полей для удаления - указываем поля, которые будем удалять, в т.ч. целевую переменную (она в переменной y). Не забываем удалять и в тестовом наборе

In [44]:
columns_to_drop = ['satisfaction','line num', 'id']

In [45]:
data = data.drop(columns_to_drop, axis=1)

In [46]:
import ColumnTransformerWithNames as ct

Создаем списки столбцов для OrdinalEncoder и для OneHotEncoder. Создаем ColumnTransformerWithNames

In [47]:
ordinalColumns = ['Gender','Customer Type','Type of Travel']
oneHotColumn = ['Class']

mlb = preprocessing.OrdinalEncoder()
dd0 = mlb.fit_transform(data[ordinalColumns])
data[ordinalColumns] = p.DataFrame(dd0)
data.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
0,1.0,0.0,13,1.0,Eco Plus,460,3,4,3,1,...,5,5,4,3,4,4,5,5,25,18.0
1,1.0,1.0,25,0.0,Business,235,3,2,3,3,...,1,1,1,5,3,1,4,1,1,6.0
2,0.0,0.0,26,0.0,Business,1142,2,2,2,2,...,5,5,4,3,4,4,4,5,0,0.0
3,0.0,0.0,25,0.0,Business,562,2,5,5,5,...,2,2,2,5,3,1,4,2,11,9.0
4,1.0,0.0,61,0.0,Business,214,3,3,3,3,...,5,3,3,4,4,3,3,3,0,0.0


In [48]:
ohe = preprocessing.OneHotEncoder()
dd = ohe.fit_transform(data[oneHotColumn].values.reshape(-1, 1)).toarray()
feature_labels = ohe.categories_
feature_labels = np.concatenate(feature_labels)
print(feature_labels)
data[feature_labels] = p.DataFrame(dd)
data=data.drop(['Class'],axis=1)
data.head()

['Business' 'Eco' 'Eco Plus']


,Gender,Customer Type,Age,Type of Travel,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,...,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,Business,Eco,Eco Plus
0,1.0,0.0,13,1.0,460,3,4,3,1,5,...,3,4,4,5,5,25,18.0,0.0,0.0,1.0
1,1.0,1.0,25,0.0,235,3,2,3,3,1,...,5,3,1,4,1,1,6.0,1.0,0.0,0.0
2,0.0,0.0,26,0.0,1142,2,2,2,2,5,...,3,4,4,4,5,0,0.0,1.0,0.0,0.0
3,0.0,0.0,25,0.0,562,2,5,5,5,2,...,5,3,1,4,2,11,9.0,1.0,0.0,0.0
4,1.0,0.0,61,0.0,214,3,3,3,3,4,...,4,4,3,3,3,0,0.0,1.0,0.0,0.0


In [66]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Создаём конвейер, где первый шаг - созданный columnTransformer, а в качестве оценщика будет выступать случайный лес. Делаем поиск по сетке параметра n_estimators - количество оценщиков в ансамбле, max_dept - максимальная глубина дерева, max_features - количество признаков для разбиения (поиск по сетке сохраняем в переменную grid_search). Для ускорения можно воспользоваться RandomizedSearchCV

In [52]:
modl = RandomForestClassifier()
modl.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [64]:
X = np.array(data)
X

array([[ 1.,  0., 13., ...,  0.,  0.,  1.],
       [ 1.,  1., 25., ...,  1.,  0.,  0.],
       [ 0.,  0., 26., ...,  1.,  0.,  0.],
       ...,
       [ 1.,  1., 30., ...,  1.,  0.,  0.],
       [ 0.,  1., 22., ...,  0.,  1.,  0.],
       [ 1.,  0., 27., ...,  1.,  0.,  0.]])

In [76]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.15, random_state=12)

In [89]:
est=[100]
depth=[15,25,35]
feat=[18]

params={'n_estimators': est, 'max_depth': depth, 'max_features': feat}

grid_search = GridSearchCV(modl, params, cv=5, 
                            return_train_score=True)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [35], 'max_features': [18],
                         'n_estimators': [100]},
             return_train_score=True)

In [90]:
res = (
    p.DataFrame({
        "mean_test_score": grid_search.cv_results_["mean_test_score"],
        "mean_fit_time": grid_search.cv_results_["mean_fit_time"]})
      .join(p.json_normalize(grid_search.cv_results_["params"]).add_prefix("param_"))
)

In [91]:
res

,mean_test_score,mean_fit_time,param_max_depth,param_max_features,param_n_estimators
0,0.963518,24.248475,35,18,100


In [92]:
0.963360	10.079459	35	15	50

SyntaxError: invalid syntax (864828319.py, line 1)

отображаем результаты поиска по сетке, отсортированные по убыванию оценки на кросс-валидации

In [88]:
res.sort_values(by='mean_test_score', ascending=False)

,mean_test_score,mean_fit_time,param_max_depth,param_max_features,param_n_estimators
5,0.963360,10.079459,35,15,50
4,0.963280,7.155748,35,10,50
2,0.963043,10.563170,25,15,50
1,0.962522,7.630497,25,10,50
3,0.960087,3.761784,35,4,50
0,0.959646,4.017543,25,4,50


In [93]:
y_pred = grid_search.predict(X_test)

from sklearn import metrics
metrics.accuracy_score(y_test, y_pred)

0.9631720775054536

Создаем DataFrame на основе названий столбцов и значений важностей признаков, выводим с убывающей сортировкой по важности

Какие факторы более значимы для оценки (ТОП3)?

Какие факторы менее значимы для оценки (ТОП3)?

Влияет ли пол респондента на оценку перелета?

In [95]:
fe = pd.DataFrame({'feat': data.columns, 'imp': grid_search.best_estimator_
.feature_importances_})
#print([(x,y) for x,y in zip(data.columns, rfc.feature_importances_)])

fe.sort_values(by=['imp'], ascending=False)

,feat,imp
10,Online boarding,0.315248
5,Inflight wifi service,0.180536
3,Type of Travel,0.103317
21,Business,0.078666
12,Inflight entertainment,0.043227
1,Customer Type,0.031607
2,Age,0.026556
4,Flight Distance,0.025275
16,Checkin service,0.024852
14,Leg room service,0.022533
